In [4]:
from utils import *
from model import *

# 加载数据
filename = 'dataset/dga_domains_full.csv'
file_dataframe = read_file(filename)
# 1表示DGA域名 0表示正常合法域

In [5]:
import nltk

# 数据向量化
list_Domain = file_dataframe['Domain'].tolist()
list_Label = file_dataframe['Label'].tolist()
# 返回数据矩阵，数据标签，最大单个域名长度，词字典
# data_vector, list_label, max_data_len, valid_chars = domain_to_vector(list_Domain, list_Label)


# 生成Bigram
Bigram_Data = []

for i in range(len(list_Domain)):
    var = []
    for j in range(len(list_Domain[i]) - 1):
        var.append(list_Domain[i][j] + list_Domain[i][j + 1])
    Bigram_Data.append(var)

Bigram_D = []

for i in range(len(list_Domain)):
    for j in range(len(list_Domain[i]) - 1):
        var = list_Domain[i][j] + list_Domain[i][j + 1]
        Bigram_D.append(var)

# 使用nltk统计bigram出现的次数

freq_dist_Data = nltk.FreqDist(Bigram_D)

# 根号处理

In [6]:
# 用bigram出现的频率替换为数字
def data_to_vector1(freq_dist, Bigram_list, list_Label):
    # 使用构造好的字典对域名进行处理
    list_data = [[freq_dist[y] for y in x] for x in Bigram_list]
    max_data_len = len(max(Bigram_list, key=len, default=''))
    # 以里面域名最大长度构造特征，小于最大长度的用一个非常小的值填充
    PAD_VALUE = 1e-10
    data_vector = sequence.pad_sequences(list_data, maxlen=max_data_len, dtype=np.float, value=PAD_VALUE)
    # 开平方根处理，使域名向量不会过大而导致无法输入神经网络，加60能够保留出现次数低的特征
    data_vector = np.sqrt(data_vector + 60) - np.sqrt(PAD_VALUE + 60)
    # 将标签列表转化为ndarray
    list_label = np.array(list_Label)
    return data_vector, list_label,max_data_len

data_vector1, list_label, max_data_len = data_to_vector1(freq_dist_Data, Bigram_Data, list_Label)
data_vector1

array([[  0.        ,   0.        ,   0.        , ...,  65.34302017,
         71.63917046,  18.84350491],
       [  0.        ,   0.        ,   0.        , ..., 141.72309365,
        129.12623648,  81.58487904],
       [  0.        ,   0.        ,   0.        , ..., 202.81518832,
        166.32585756, 122.2578794 ],
       ...,
       [  0.        ,   0.        ,   0.        , ..., 157.00232617,
        137.15744942,  95.69483764],
       [  0.        ,   0.        ,   0.        , ..., 146.36113892,
         57.96552393,  54.99955108],
       [  0.        ,   0.        ,   0.        , ...,  86.2274338 ,
         80.62823769, 151.14393229]])

# 对数处理

In [7]:
# 用bigram出现的频率替换为数字
def data_to_vector2(freq_dist, Bigram_list, list_Label):
    # 使用构造好的字典对域名进行处理
    list_data = [[freq_dist[y] for y in x] for x in Bigram_list]
    max_data_len = len(max(Bigram_list, key=len, default=''))
    # 以里面域名最大长度构造特征，小于最大长度的用一个非常小的值填充
    PAD_VALUE = 1e-10
    data_vector = sequence.pad_sequences(list_data, maxlen=max_data_len, dtype=np.float, value=PAD_VALUE)
    # 对数处理
    data_vector = np.log(data_vector + 60) - np.log(PAD_VALUE + 60)
    # 将标签列表转化为ndarray
    list_label = np.array(list_Label)
    return data_vector, list_label,max_data_len

data_vector2, list_label, max_data_len = data_to_vector2(freq_dist_Data, Bigram_Data, list_Label)
data_vector2

array([[0.        , 0.        , 0.        , ..., 4.48901083, 4.65427776,
        2.4666861 ],
       [0.        , 0.        , 0.        , ..., 5.91983427, 5.74375077,
        4.89034913],
       [0.        , 0.        , 0.        , ..., 6.6052077 , 6.22459143,
        5.64078351],
       ...,
       [0.        , 0.        , 0.        , ..., 6.11449306, 5.85779029,
        5.18365446],
       [0.        , 0.        , 0.        , ..., 5.98095114, 4.27620305,
        4.18382973],
       [0.        , 0.        , 0.        , ..., 4.99167898, 4.86881568,
        6.04207844]])

# 原向量

In [8]:
# 用bigram出现的频率替换为数字
def data_to_vector3(freq_dist, Bigram_list, list_Label):
    # 使用构造好的字典对域名进行处理
    list_data = [[freq_dist[y] for y in x] for x in Bigram_list]
    max_data_len = len(max(Bigram_list, key=len, default=''))
    # 以里面域名最大长度构造特征，小于最大长度的用一个非常小的值填充
    data_vector = sequence.pad_sequences(list_data, maxlen=max_data_len, dtype=np.float)
    # 将标签列表转化为ndarray
    list_label = np.array(list_Label)
    return data_vector, list_label,max_data_len
data_vector3, list_label, max_data_len = data_to_vector3(freq_dist_Data, Bigram_Data, list_Label)
data_vector3

array([[    0.,     0.,     0., ...,  5282.,  6242.,   647.],
       [    0.,     0.,     0., ..., 22281., 18674.,  7920.],
       [    0.,     0.,     0., ..., 44276., 30241., 16841.],
       ...,
       [    0.,     0.,     0., ..., 27082., 20937., 10640.],
       [    0.,     0.,     0., ..., 23689.,  4258.,  3877.],
       [    0.,     0.,     0., ...,  8771.,  7750., 25186.]])

# 标准归一化处理

In [10]:
from sklearn.preprocessing import StandardScaler,MinMaxScaler
scaler = StandardScaler()
minmax = MinMaxScaler()
data_vector4 = minmax.fit_transform(scaler.fit_transform(data_vector3))
data_vector4

array([[0.        , 0.        , 0.        , ..., 0.07039194, 0.08317341,
        0.00849028],
       [0.        , 0.        , 0.        , ..., 0.29693351, 0.24885388,
        0.10542871],
       [0.        , 0.        , 0.        , ..., 0.59005557, 0.40300656,
        0.22433257],
       ...,
       [0.        , 0.        , 0.        , ..., 0.36091528, 0.27901274,
        0.14168233],
       [0.        , 0.        , 0.        , ..., 0.31569759, 0.05673277,
        0.05154145],
       [0.        , 0.        , 0.        , ..., 0.116889  , 0.10327043,
        0.3355592 ]])